In [18]:
from nba_api.stats.endpoints import teamdashlineups
import pandas as pd
import time

In [2]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()

In [3]:
team_dict = {}
for i in nba_teams:
    team_name = i['full_name']
    team_id = i['id']
    team_dict[team_name]= team_id

In [21]:
def get_lineups(team_id_i, retries=3, delay=2):
    import time
    
    for attempt in range(retries):
        try:
            lineup = teamdashlineups.TeamDashLineups(
                date_from_nullable = "",
                date_to_nullable = "",
                game_id_nullable = "",
                game_segment_nullable = "",
                group_quantity = 5,
                last_n_games = 0,
                league_id_nullable = "00",
                location_nullable = "",
                measure_type_detailed_defense = "Base",
                month = 0,
                opponent_team_id = 0,
                outcome_nullable = "",
                pace_adjust = "N",
                plus_minus = "N",
                po_round_nullable = "",
                per_mode_detailed = "Totals",
                period = 0,
                rank = "N",
                season = "2023-24",
                season_segment_nullable = "",
                season_type_all_star = "Regular Season",
                shot_clock_range_nullable = "",
                team_id = team_id_i,
                vs_conference_nullable = "",
                vs_division_nullable = ""
            )
            
            df = lineup.get_data_frames()
            all_lineups = df[1]
            return all_lineups
            
        except Exception as e:
            if attempt < retries - 1:  # if not last attempt
                time.sleep(delay)  # wait before retrying
                continue
            else:
                print(f"Failed to get data for team_id: {team_id_i}")
                print(f"Error: {str(e)}")
                return None

In [22]:
# Modified main loop
league_lineup = pd.DataFrame()

for i in team_dict:
    team_id_i = team_dict[i]
    team_lineup = get_lineups(team_id_i)
    
    if team_lineup is not None:
        team_lineup['team'] = i
        team_lineup['team_id'] = team_id_i
        league_lineup = pd.concat([league_lineup, team_lineup], ignore_index=True)
    
    time.sleep(1)  # Add delay between requests

Failed to get data for team_id: 1610612744
Error: Expecting value: line 1 column 1 (char 0)


In [23]:
league_lineup['players_list'] = league_lineup['GROUP_NAME'].str.split(' - ')
league_lineup = league_lineup.sort_values(by='team')
league_lineup.to_csv('NBALineup2023.csv')